<div align='center'>
<img src="..\..\..\01_aux_files\img\Video_Game_Sales\1.png" height='50%' width='30%'/>
</div>




<h2>Project idea and data sources: <a href='https://www.linkedin.com/in/fabian-werkmeister/'>Fabian Werkmeister (LinkedIn)</a></h2>
<h3>Project's full material from Damian's notion: <a href='https://fabianwerkmeister.notion.site/Video-Game-Sales-8249d8cddf5c44fbb58712d950a52a91'>Click Here!</a></h3>

<h1 align='center'>FULL ETL</h1>

#### We will host our database on a local SQL Server DataWarehouse, updating and inserting data. We will refresh our data --Hopefully-- on a daily basis, and we will show the results of the Data Analytics track with Power BI.
#### ¡¡¡Even when our local server is not reacheble from outside, we won't save our credentials in the actual code, just as a good practice!!!

## Needed Libraries

In [18]:
import pandas as pd
from sqlalchemy import create_engine, text, MetaData, Table, select

## Loading data on SRC schema, scrapping data from web

!!!<br/>
If you checked https://www.vgchartz.com/gamedb/ table, you'd probably realize that there are some records without a Last Updated date. We are not able to bring them without scanning the whole result set, because you can't filter Last Update Date value from URL.
So, we are bringing them on full loads only (running a full load everyday in a production environment would be expensive).
On Delta (incremental) refreshes, we will bring just the latest 1000 updated/added records, because we can sort results by date.<br/>
!!!

In [ ]:
PMODE = "FULL"
%run "Web_Scrapping_And_Pandas/web_Scrapping_01.ipynb"

In [ ]:
directory = '../../../01_aux_files/results/Video_Game_Sales/Web Scrapping & Pandas/'
%run "Web_Scrapping_And_Pandas/web_Scrapping_02.ipynb"

### Needed variables

In [19]:
user_name = 'XXXXX' ## We won't save credentials on github! 
password =  'XXXXX' ## We won't save credentials on github!
dw_server = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
db_name = 'VideoGames_Sales'

SRC_Tables = ['Developer', 'Publisher', 'VideoGames']

connection_string = (
    f'mssql+pyodbc://{user_name}:{password}@{dw_server}/'
    f'{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
)

# Connecting with SQL Server
engine = create_engine(connection_string, fast_executemany=True)

## Loading dimensions that go from SRC to DW without and INTERMEDIATE table

In [5]:
with engine.connect() as conn:
    for table_name in SRC_Tables:
        try:
            conn.execute(text(f"exec DW.SP_Load_Dim_{table_name}"))
            print(f"Dim_{table_name} Successfully loaded!")
            conn.commit()
        except Exception as e:
            raise ValueError(str(e))

Dim_Developer Successfully loaded!
Dim_Publisher Successfully loaded!
Dim_VideoGames Successfully loaded!


## Loading Dim_Consoles_Information, that goes from SRC -> INTERMEDIATE -> DW

In [6]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        conn.execute(text(f"exec DW.SP_Load_Intermediate_Dim_Consoles_Information"))
        print(f"INTERMEDIATE.Dim_Consoles_Information Successfully loaded!")

        ## INTERMEDIATE -> DW
        conn.execute(text(f"exec DW.SP_Load_Dim_Consoles_Information"))
        print(f"DW.Dim_Consoles_Information Successfully loaded!")

        conn.commit()
    except Exception as e:
        raise ValueError(str(e))

INTERMEDIATE.Dim_Consoles_Information Successfully loaded!
DW.Dim_Consoles_Information Successfully loaded!


## As a final step, we will load our Fact_VideoGames_Sales, that goes from SRC -> INTERMEDIATE -> DW

In [7]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        conn.execute(text(f"exec DW.SP_Load_Intermediate_Fact_VideoGames_Sales"))
        print(f"INTERMEDIATE.Fact_VideoGames_Sales Successfully loaded!")

        ## INTERMEDIATE -> DW
        conn.execute(text(f"exec DW.SP_Load_Fact_VideoGames_Sales"))
        print(f"DW.Fact_VideoGames_Sales Successfully loaded!")

        conn.commit()
    except Exception as e:
        raise ValueError(str(e))

INTERMEDIATE.Fact_VideoGames_Sales Successfully loaded!
DW.Fact_VideoGames_Sales Successfully loaded!


<h1 align = 'center'>Showing Dimensions, Facts and results</h1>
<h2 align = 'center'>(Please, check scripts to realize how are we loading them)</h2>
<h3 align = 'center'>¡¡¡TIMEZONE: ARGENTINA!!!</h3>

## Dimensions
Every Dimension have an unique key defined, so, We are 100% that It won't insert duplicated values. Otherwise, It would Fail
It also ensures that we won't have duplicated records when joining data from fact to our dimensions

### Dim_VideoGames

In [8]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_VideoGames")).fetchall()
        Dim_VideoGames = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_VideoGames.sample(10)

,ID_VideoGame,Game_Name,Audit_Insert_TS,Audit_Insert_Username
35307,35307,SUIGETSU PORTABLE,2024-01-11 19:49:48.570,ETL System
6588,6588,NINTENDO LABO: TOY-CON 03 VEHICLE KIT,2024-01-11 19:49:48.570,ETL System
36959,36959,THE JETSONS: ROBOT PANIC,2024-01-11 19:49:48.570,ETL System
38698,38698,UGOITE ASOBU BOXING,2024-01-11 19:49:48.570,ETL System
9799,9799,DATABASETESTINGEU,2024-01-11 19:49:48.570,ETL System
3027,3027,FIFA 12,2024-01-11 19:49:48.570,ETL System
34933,34933,STAR WARS: ROGUE SQUADRON,2024-01-11 19:49:48.570,ETL System
18217,18217,ENKAKU SOUSA: SANA E NO 23 HIAI,2024-01-11 19:49:48.570,ETL System
32060,32060,RYOUSHUU JOKYOUSHI: NIKUYOKU NO HOUKAGO,2024-01-11 19:49:48.570,ETL System
25288,25288,MAGICIAN'S QUEST: MYSTERIOUS TIMES,2024-01-11 19:49:48.570,ETL System


### Dim_Developer

In [9]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_Developer")).fetchall()
        Dim_Developer = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_Developer.sample(10)

,ID_Developer,Developer,Audit_Insert_TS,Audit_Insert_Username
136,136,ARC SYSTEM WORKS,2024-01-11 19:49:48.460,ETL System
8029,8029,DIMPS CORPORATION / AMUSEMENT VISION,2024-01-11 19:49:48.460,ETL System
4644,4644,LES BIRD AND RUBEN CABRERA,2024-01-11 19:49:48.460,ETL System
1278,1278,UBISOFT QUEBEC,2024-01-11 19:49:48.460,ETL System
3460,3460,ONTECA,2024-01-11 19:49:48.460,ETL System
1677,1677,ZHAIJIDIAN,2024-01-11 19:49:48.460,ETL System
834,834,WORLDWALKER GAMES,2024-01-11 19:49:48.460,ETL System
3765,3765,IGG.COM,2024-01-11 19:49:48.460,ETL System
6797,6797,NINTENDO EAD / RETRO STUDIOS,2024-01-11 19:49:48.460,ETL System
4013,4013,BIT MANAGERS,2024-01-11 19:49:48.460,ETL System


### Dim_Publisher

In [10]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_Publisher")).fetchall()
        Dim_Publisher = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_Publisher.sample(10)

,ID_Publisher,Publisher,Audit_Insert_TS,Audit_Insert_Username
350,350,FATSHARK,2024-01-11 19:49:48.506666,ETL System
568,568,THE FARM 51,2024-01-11 19:49:48.506666,ETL System
1985,1985,BIT CORPORATION,2024-01-11 19:49:48.506666,ETL System
3086,3086,GAMES FACTORY ONLINE,2024-01-11 19:49:48.506666,ETL System
765,765,"ARCEN GAMES, LLC",2024-01-11 19:49:48.506666,ETL System
2653,2653,NEW WORLD,2024-01-11 19:49:48.506666,ETL System
1965,1965,MEDIA GROUP,2024-01-11 19:49:48.506666,ETL System
1679,1679,HUDSON ENTERTAINMENT,2024-01-11 19:49:48.506666,ETL System
538,538,ROCKETPUNCH GAMES,2024-01-11 19:49:48.506666,ETL System
1285,1285,MAGES,2024-01-11 19:49:48.506666,ETL System


### Dim_Consoles_Information

In [11]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_Consoles_Information")).fetchall()
        Dim_Consoles_Information = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_Consoles_Information.sample(10)

,ID_Console,Console_Abbreviation,Console_Name,Developer,Release_Date,Origin_Country,Generation,Type,Media_Type,Graphics,Online_Play,Predecessor,Successor,Audit_Insert_TS,Audit_Insert_Username,Audit_Update_TS,Audit_Update_Username,INCOMPLETE_INFORMATION
101,101,VC,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,NaT,None,YES
42,42,TE,TURBOEXPRESS,NEC,1990-12-01,JAPAN,FIFTH,HANDHELD,HUCARD,16-BIT,NO,TG16,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,2024-01-24 22:57:19.703333,ETL System,NO
103,103,AMIG,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,NaT,None,YES
74,74,GP32,GAMEPARK 32,GAMEPARK,2001-11-23,KOREA,SIXTH,HANDHELD,SMART MEDIA,32-BIT,NO,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,2024-01-24 22:57:19.703333,ETL System,NO
64,64,TT,TOMY TUTOR,TOMY,1982-07-01,JAPAN,SECOND,HOME CONSOLE,CARTRIDGE,8-BIT,NO,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,2024-01-24 22:57:19.703333,ETL System,NO
14,14,XONE,XBOX ONE,MICROSOFT,2013-11-22,USA,EIGHTH,HOME CONSOLE,PHYSICAL/DIGITAL,1080P,YES,XBOX 360,XBOX SERIES X/S,2024-01-11 19:49:49.730,ETL System,2024-01-24 22:57:19.703333,ETL System,NO
6,6,PS,PLAYSTATION,SONY,1994-12-03,JAPAN,FIFTH,HOME CONSOLE,PHYSICAL,32-BIT,NO,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,2024-01-24 22:57:19.703333,ETL System,NO
97,97,WINP,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,NaT,None,YES
115,115,ACO,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,NaT,None,YES
12,12,3DS,NINTENDO 3DS,NINTENDO,2011-02-26,JAPAN,EIGHTH,HANDHELD,PHYSICAL/DIGITAL,3D,YES,DS,SWITCH,2024-01-11 19:49:49.730,ETL System,2024-01-24 22:57:19.703333,ETL System,NO


!!!<br/>
Note that we have some values with INCOMPLETE_INFORMATION: NO. 
This means that those values were inserted just with their abbreviations.
In a future, we could update their information, but It will work for now.<br/>
!!!

## Fact Table
It has a primary key defined over its dimensions and Audit_Insert_TS field. As It is fixed by batch, It won't insert duplicated rows for the same day. Otherwise, It would fail.

In [12]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Fact_VideoGames_Sales")).fetchall()
        Fact_VideoGames_Sales = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Fact_VideoGames_Sales.sample(10)

,ID_VideoGame,ID_Console,ID_Publisher,ID_Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Total_Sales,NA_Sales,...,Japan_Sales,Other_Sales,ID_Release_Date,Last_Update,HASH_VALUE,Audit_Insert_TS,Audit_Insert_Username,Audit_Update_TS,Audit_Update_Username,IS_CURRENT_VERSION
42803,25928,22,9,1222,None,7.9,None,NaN,250000.0,190000.0,...,NaN,10000.0,20050607,None,b'y\x1d\x0b\x9a\xd4\x9f\x99\xec\x11\xd6U\xa7w\...,2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
40248,24054,2,1794,3899,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20090326,None,"b'j6\x8d2\xc4\xcb\xa0\x03,\xf1:\xe0\xc8l@\x90\...",2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
51218,31940,89,2210,7674,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20140430,None,b'\xf6\xba\xd2q\xbb\x13$A\xcf\xd7lg\xff\xdeI\x...,2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
31198,17237,19,1454,3249,None,None,None,NaN,NaN,NaN,...,NaN,NaN,19830101,None,"b""\xaa\xf3m#\xdd\x1b\xfdKz-\xf1m6\xbd'\x990g^\...",2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
43998,26793,9,9,5024,None,None,None,NaN,590000.0,340000.0,...,NaN,60000.0,20081020,None,b'\xdd6\xd5U4\xeb\xd38\x92T\xbb\xad?F\x83g;\xf...,2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
28226,15061,35,1795,2751,None,8.3,None,NaN,NaN,NaN,...,NaN,NaN,20041115,None,"b'\x7f\xc9\xee#\x0c\xa8\xe6\x08""A\xb0\x02\x1c\...",2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
28901,15538,28,68,3105,None,None,None,NaN,NaN,NaN,...,NaN,NaN,19970526,None,b'\xe9j\xba\x15+\x0ff-\x81\xd5J\xc4\xc6\xeb\xd...,2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
30032,16377,84,288,5237,None,None,None,NaN,NaN,NaN,...,NaN,NaN,19850101,None,b'o\xadV2}\x91\xabT\xee\x03\x1d\xecO\x84\xb2\x...,2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
8172,2926,99,67,6162,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20090930,None,"b'""v\xd3\x11\t\xf7#\x02\xd3V""2I\xc6\x84A\xe1\x...",2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION
9220,3235,13,0,154,None,None,None,NaN,NaN,NaN,...,NaN,NaN,0,2020-08-26,b'\xa0X\xa6\x9d~1\x1e\xc7\r\x1e\xd2\xd4D\x87\x...,2024-01-11 19:49:49.570,ETL System,NaT,None,CURRENT VERSION


### Full Data Report Sample

In [13]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text(
        """
        select
		 VIDEOGAMES.Game_Name
		,CONSOLES.Console_Abbreviation
		,CONSOLES.Console_Name
		,CONSOLES.Generation AS Concole_Generation
		,Publisher.Publisher
		,Developer.Developer
		,VGChartz_Score
		,Critic_Score
		,User_Score
		,Total_Shipped
		,Total_Sales
		,NA_Sales
		,PAL_Sales
		,Japan_Sales
		,Other_Sales
		,DDATE.CalendarDate AS Release_Date
		,DDATE.CalendarYear AS Release_Date_Year
		,DDATE.CalendarMonthName AS Release_Date_Month
		,DDATE.FiscalDayOfMonth Release_Date_Fiscal_Day_Of_Month
	from 
		DW.Fact_VideoGames_Sales FACT
		left join DW.Dim_VideoGames VIDEOGAMES ON VIDEOGAMES.ID_VideoGame = FACT.ID_VideoGame
		left join DW.Dim_Consoles_Information CONSOLES ON CONSOLES.ID_Console = FACT.ID_Console
		left join DW.Dim_Publisher PUBLISHER ON PUBLISHER.ID_Publisher = FACT.ID_Publisher
		left join DW.Dim_Developer DEVELOPER ON DEVELOPER.ID_Developer = FACT.ID_Developer
		left join DW.Dim_Date DDATE ON DDATE.DateKey = FACT.ID_Release_Date
        """)).fetchall()
        df = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

df.sample(10)

,Game_Name,Console_Abbreviation,Console_Name,Concole_Generation,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Total_Sales,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Release_Date,Release_Date_Year,Release_Date_Month,Release_Date_Fiscal_Day_Of_Month
16048,SLAYAWAY CAMP: BUTCHER'S CUT,NS,NINTENDO SWITCH,EIGHTH,DIGERATI,DIGERATI,None,None,8.9,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-22,2018,March,22
3884,CALL OF DUTY: BLACK OPS,DS,NINTENDO DS,SEVENTH,ACTIVISION,N-SPACE,None,7.0,None,NaN,590000.0,500000.0,40000.0,NaN,40000.0,2010-11-09,2010,November,9
29700,FORSAKEN,PC,NOT SPECIFIED,NOT SPECIFIED,ACCLAIM ENTERTAINMENT,PROBE ENTERTAINMENT LIMITED,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,1998-04-30,1998,April,30
34186,FROBOT,PC,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,FUGAZO,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,9999-12-31,0,NOT SPECIFIED,0
59808,VIKING BROTHERS,OSX,NOT SPECIFIED,NOT SPECIFIED,VIVA MEDIA,NOT SPECIFIED,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-23,2014,April,23
53799,SHINSETSU SAMURAI SPIRITS: BUSHIDOU RETSUDEN,SAT,SEGA SATURN,FIFTH,SNK,SNK CORPORATION,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,1997-06-27,1997,June,27
4992,ENTER THE GUNGEON,PS4,PLAYSTATION 4,EIGHTH,DEVOLVER DIGITAL,DODGE ROLL,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-05,2016,April,5
56625,TOTAL ECLIPSE,3DO,3DO INTERACTIVE MULTIPLAYER,FIFTH,CRYSTAL DYNAMICS,CRYSTAL DYNAMICS,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,1993-01-01,1993,January,1
25236,VESTA,XONE,XBOX ONE,EIGHTH,FINALBOSS GAMES,FINALBOSS GAMES,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-19,2018,January,19
5127,FROZEN SYNAPSE,ALL,NOT SPECIFIED,NOT SPECIFIED,MODE 7,MODE 7 GAMES,None,None,None,450000.0,NaN,NaN,NaN,NaN,NaN,2011-05-26,2011,May,26


### Showing Today's inserted/updated rows by entity

In [14]:
query = """
DECLARE @TODAY DATE = CAST(GETDATE() AS DATE)

SELECT
	'Dim_VideoGames' as Entity,
	COUNT(1) Total_Inserted_Rows,
	'Entity does not update data' as Total_Updated_Rows
FROM
	DW.Dim_VideoGames
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Dim_Developer' as Entity,
	COUNT(1) Total_Inserted_Rows,
	'Entity does not update data' as Total_Updated_Rows
FROM
	DW.Dim_Developer
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Dim_Publisher' as Entity,
	COUNT(1) Total_Inserted_Rows,
	'Entity does not update data' as Total_Updated_Rows
FROM
	DW.Dim_Publisher
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Dim_Consoles_Information' as Entity,
	SUM(CASE WHEN Audit_Update_TS IS NULL THEN 1 ELSE 0 END) Total_Inserted_Rows,
	CAST(SUM(CASE WHEN Audit_Update_TS IS NOT NULL THEN 1 ELSE 0 END) AS nvarchar(20)) as Total_Updated_Rows
FROM
	DW.Dim_Consoles_Information
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY OR CAST(Audit_Update_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Fact_VideoGames_Sales' as Entity,
	SUM(CASE WHEN Audit_Update_TS IS NULL THEN 1 ELSE 0 END) Total_Inserted_Rows,
	CAST(SUM(CASE WHEN Audit_Update_TS IS NOT NULL THEN 1 ELSE 0 END) AS nvarchar(20)) as Total_Updated_Rows
FROM
	DW.Fact_VideoGames_Sales
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY OR CAST(Audit_Update_TS AS DATE) = @TODAY
"""

with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text(query)).fetchall()
        df = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

df.sort_values(by=['Entity'], ascending=False)

,Entity,Total_Inserted_Rows,Total_Updated_Rows
4,Fact_VideoGames_Sales,1,1
0,Dim_VideoGames,0,Entity does not update data
2,Dim_Publisher,0,Entity does not update data
1,Dim_Developer,0,Entity does not update data
3,Dim_Consoles_Information,0,92


<h1 align ='center'>!!01/24/2024 UPDATE: HISTORY AVAILABLE!!</h1>
<h3>I received some videogame records that already existed in our fact table, so now we are able to watch the evolution of the different facts over history!</h3> 
<h3>This is prooves that our updating/history tracking logic works fine! Let's check them :D</h3>

In [27]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text(
        """
WITH CTE_GAMES_WITH_CHANGES
AS
(
    SELECT 
        ID_VIDEOGAME,
        ID_Console,
        ID_Publisher,
        ID_Developer,
        COUNT(distinct IS_CURRENT_VERSION) AS VERSIONS
    FROM 
        DW.Fact_VideoGames_Sales
    GROUP BY
        ID_VIDEOGAME,
        ID_Console,
        ID_Publisher,
        ID_Developer
    HAVING
        COUNT(distinct IS_CURRENT_VERSION) > 1

)

SELECT VIDEOGAMES.Game_Name
      ,CONSOLES.Console_Abbreviation
      ,CONSOLES.Console_Name
      ,PUBLISHER.Publisher
      ,DEVELOPER.Developer
      ,FACT.VGChartz_Score
      ,FACT.Critic_Score
      ,FACT.User_Score
      ,FACT.Total_Shipped
      ,FACT.Total_Sales
      ,FACT.NA_Sales
      ,FACT.PAL_Sales
      ,FACT.Japan_Sales
      ,FACT.Other_Sales
      ,FACT.ID_Release_Date
      ,FACT.Audit_Insert_TS
      ,FACT.Audit_Update_TS
      ,FACT.IS_CURRENT_VERSION
  FROM 
		DW.Fact_VideoGames_Sales FACT
        inner join CTE_GAMES_WITH_CHANGES CTE --TO KEEP JUST UPDATED RECORDS
        ON  FACT.ID_VIDEOGAME = CTE.ID_VIDEOGAME 
        AND FACT.ID_Console   = CTE.ID_Console  
        AND FACT.ID_Publisher = CTE.ID_Publisher
        AND FACT.ID_Developer = CTE.ID_Developer
		left join DW.Dim_VideoGames VIDEOGAMES ON VIDEOGAMES.ID_VideoGame = FACT.ID_VideoGame
		left join DW.Dim_Consoles_Information CONSOLES ON CONSOLES.ID_Console = FACT.ID_Console
		left join DW.Dim_Publisher PUBLISHER ON PUBLISHER.ID_Publisher = FACT.ID_Publisher
		left join DW.Dim_Developer DEVELOPER ON DEVELOPER.ID_Developer = FACT.ID_Developer
		left join DW.Dim_Date DDATE ON DDATE.DateKey = FACT.ID_Release_Date
  ORDER BY
        VIDEOGAMES.Game_Name, 
        CONSOLES.Console_Abbreviation, 
        PUBLISHER.Publisher, 
        DEVELOPER.Developer,
        FACT.Audit_Insert_TS ASC
        """)).fetchall()
        df = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

df

,Game_Name,Console_Abbreviation,Console_Name,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Total_Sales,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,ID_Release_Date,Audit_Insert_TS,Audit_Update_TS,IS_CURRENT_VERSION
0,PALWORLD,ALL,NOT SPECIFIED,POCKETPAIR,POCKETPAIR,None,None,None,1000000.0,None,None,None,None,None,20240119,2024-01-20 19:31:40.586666,2024-01-23 21:41:34.450,NON CURRENT VERSION
1,PALWORLD,ALL,NOT SPECIFIED,POCKETPAIR,POCKETPAIR,None,None,None,6000000.0,None,None,None,None,None,20240119,2024-01-23 21:41:33.450000,2024-01-24 22:57:20.940,NON CURRENT VERSION
2,PALWORLD,ALL,NOT SPECIFIED,POCKETPAIR,POCKETPAIR,None,None,None,7000000.0,None,None,None,None,None,20240119,2024-01-24 22:57:19.940000,NaT,CURRENT VERSION
3,STAR TREK ONLINE,PS4,PLAYSTATION 4,NOT SPECIFIED,CRYPTIC STUDIOS,None,None,None,NaN,None,None,None,None,None,0,2024-01-11 19:49:49.570000,2024-01-23 21:41:34.450,NON CURRENT VERSION
4,STAR TREK ONLINE,PS4,PLAYSTATION 4,NOT SPECIFIED,CRYPTIC STUDIOS,None,7.5,None,NaN,None,None,None,None,None,0,2024-01-23 21:41:33.450000,NaT,CURRENT VERSION


## We can watch the evolution of each record, and the difference between each row/version, ordered by insertion (on our DW) date

Closing connection properly

In [28]:
engine.dispose()

<h1 align='center'>Thanks For Watching! :D</h1>